<h1><center>Laboratorio 9: Optimización de modelos 💯</center></h1>

<center><strong>MDS7202: Laboratorio de Programación Científica para Ciencia de Datos - Primavera 2024</strong></center>

### **Cuerpo Docente:**

- Profesores: Ignacio Meza, Sebastián Tinoco
- Auxiliar: Eduardo Moya
- Ayudantes: Nicolás Ojeda, Melanie Peña, Valentina Rojas

### Equipo: SUPER IMPORTANTE - notebooks sin nombre no serán revisados

- Nombre de alumno 1: Cristopher Urbina H.
- Nombre de alumno 2: Joaquin Zamora O.


### **Link de repositorio de GitHub:** [Repositorio](https://github.com/CrisU8/MDS7202-Primavera2024)

### Temas a tratar

- Predicción de demanda usando `xgboost`
- Búsqueda del modelo óptimo de clasificación usando `optuna`
- Uso de pipelines.

### Reglas:

- **Grupos de 2 personas**
- Cualquier duda fuera del horario de clases al foro. Mensajes al equipo docente serán respondidos por este medio.
- Prohibidas las copias.
- Pueden usar cualquer matrial del curso que estimen conveniente.
- Código que no se pueda ejecutar, no será revisado.

### Objetivos principales del laboratorio

- Optimizar modelos usando `optuna`
- Recurrir a técnicas de *prunning*
- Forzar el aprendizaje de relaciones entre variables mediante *constraints*
- Fijar un pipeline con un modelo base que luego se irá optimizando.

El laboratorio deberá ser desarrollado sin el uso indiscriminado de iteradores nativos de python (aka "for", "while"). La idea es que aprendan a exprimir al máximo las funciones optimizadas que nos entrega `pandas`, las cuales vale mencionar, son bastante más eficientes que los iteradores nativos sobre DataFrames.

# Importamos librerias útiles

In [98]:
#!pip install -qq xgboost optuna

# El emprendimiento de Fiu

Tras liderar de manera exitosa la implementación de un proyecto de ciencia de datos para caracterizar los datos generados en Santiago 2023, el misterioso corpóreo **Fiu** se anima y decide levantar su propio negocio de consultoría en machine learning. Tras varias e intensas negociaciones, Fiu logra encontrar su *primera chamba*: predecir la demanda (cantidad de venta) de una famosa productora de bebidas de calibre mundial. Al ver el gran potencial y talento que usted ha demostrado en el campo de la ciencia de datos, Fiu lo contrata como data scientist para que forme parte de su nuevo emprendimiento.

Para este laboratorio deben trabajar con los datos `sales.csv` subidos a u-cursos, el cual contiene una muestra de ventas de la empresa para diferentes productos en un determinado tiempo.

Para comenzar, cargue el dataset señalado y visualice a través de un `.head` los atributos que posee el dataset.

<i><p align="center">Fiu siendo felicitado por su excelente desempeño en el proyecto de caracterización de datos</p></i>
<p align="center">
  <img src="https://media-front.elmostrador.cl/2023/09/A_UNO_1506411_2440e.jpg">
</p>

In [99]:
import pandas as pd
import numpy as np
from datetime import datetime

from IPython.terminal.shortcuts.filters import pass_through
from yellowbrick.contrib.wrapper import regressor

df = pd.read_csv("sales.csv")

df.head()

,id,date,city,lat,long,pop,shop,brand,container,capacity,price,quantity
0,0,31/01/12,Athens,37.97945,23.71622,672130,shop_1,kinder-cola,glass,500ml,0.96,13280
1,1,31/01/12,Athens,37.97945,23.71622,672130,shop_1,kinder-cola,plastic,1.5lt,2.86,6727
2,2,31/01/12,Athens,37.97945,23.71622,672130,shop_1,kinder-cola,can,330ml,0.87,9848
3,3,31/01/12,Athens,37.97945,23.71622,672130,shop_1,adult-cola,glass,500ml,1.00,20050
4,4,31/01/12,Athens,37.97945,23.71622,672130,shop_1,adult-cola,can,330ml,0.39,25696


## 1 Generando un Baseline (5 puntos)

<p align="center">
  <img src="https://media.tenor.com/O-lan6TkadUAAAAC/what-i-wnna-do-after-a-baseline.gif">
</p>

Antes de entrenar un algoritmo, usted recuerda los apuntes de su magíster en ciencia de datos y recuerda que debe seguir una serie de *buenas prácticas* para entrenar correcta y debidamente su modelo. Después de un par de vueltas, llega a las siguientes tareas:

1. Separe los datos en conjuntos de train (70%), validation (20%) y test (10%). Fije una semilla para controlar la aleatoriedad. [0.5 puntos]
2. Implemente un `FunctionTransformer` para extraer el día, mes y año de la variable `date`. Guarde estas variables en el formato categorical de pandas. [1 punto]
3. Implemente un `ColumnTransformer` para procesar de manera adecuada los datos numéricos y categóricos. Use `OneHotEncoder` para las variables categóricas. `Nota:` Utilice el método `.set_output(transform='pandas')` para obtener un DataFrame como salida del `ColumnTransformer` [1 punto]
4. Guarde los pasos anteriores en un `Pipeline`, dejando como último paso el regresor `DummyRegressor` para generar predicciones en base a promedios. [0.5 punto]
5. Entrene el pipeline anterior y reporte la métrica `mean_absolute_error` sobre los datos de validación. ¿Cómo se interpreta esta métrica para el contexto del negocio? [0.5 puntos]
6. Finalmente, vuelva a entrenar el `Pipeline` pero esta vez usando `XGBRegressor` como modelo **utilizando los parámetros por default**. ¿Cómo cambia el MAE al implementar este algoritmo? ¿Es mejor o peor que el `DummyRegressor`? [1 punto]
7. Guarde ambos modelos en un archivo .pkl (uno cada uno) [0.5 puntos]

In [100]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 7456 entries, 0 to 7455
Data columns (total 12 columns):
 #   Column     Non-Null Count  Dtype  
---  ------     --------------  -----  
 0   id         7456 non-null   int64  
 1   date       7456 non-null   object 
 2   city       7456 non-null   object 
 3   lat        7456 non-null   float64
 4   long       7456 non-null   float64
 5   pop        7456 non-null   int64  
 6   shop       7456 non-null   object 
 7   brand      7456 non-null   object 
 8   container  7456 non-null   object 
 9   capacity   7456 non-null   object 
 10  price      7456 non-null   float64
 11  quantity   7456 non-null   int64  
dtypes: float64(3), int64(3), object(6)
memory usage: 699.1+ KB


In [101]:
from sklearn import set_config
set_config(transform_output="pandas")

In [102]:
from sklearn.model_selection import train_test_split

# Separar en train (70%) y dev (30%)
train_data, dev_data = train_test_split(df, test_size=0.3, random_state=42)

# Separar temp en validation (20%) y test (10%)
validation_data, test_data = train_test_split(dev_data, test_size=0.33, random_state=42)

# Ver tamaños
print(f'Tamaño train: {train_data.shape[0]}, validation: {validation_data.shape[0]}, test: {test_data.shape[0]}')


Tamaño train: 5219, validation: 1498, test: 739


In [103]:
from sklearn.preprocessing import FunctionTransformer

# Función para extraer día, mes, y año
def extract_date_features(df):
    # Especificar el formato de la fecha como 'DD/MM/YY'
    df['day'] = pd.to_datetime(df['date'], format='%d/%m/%y').dt.day.astype(int)
    df['month'] = pd.to_datetime(df['date'], format='%d/%m/%y').dt.month.astype(int)
    df['year'] = pd.to_datetime(df['date'], format='%d/%m/%y').dt.year.astype(int)
    return df[['day', 'month', 'year']]

# Implementar el FunctionTransformer
date_transformer = FunctionTransformer(extract_date_features)

# Aplicar el transformer
date_features = date_transformer.fit_transform(train_data)
date_features.head()

,day,month,year
292,30,4,2012
3366,28,2,2015
3685,30,6,2015
2404,30,4,2014
2855,30,9,2014


In [104]:
from sklearn.compose import ColumnTransformer
from sklearn.preprocessing import OneHotEncoder, StandardScaler, FunctionTransformer

# Identificar las columnas categóricas y numéricas
categorical_cols = ['city', 'shop', 'brand', 'container', 'capacity']
numerical_cols = ['lat', 'long', 'price']

# Crear el ColumnTransformer con el transformer para la fecha y OneHotEncoder
preprocessor = ColumnTransformer(
    transformers=[
        ('num', StandardScaler(), numerical_cols),
        ('cat', OneHotEncoder(sparse_output=False), categorical_cols),
        ('date', date_transformer, ['date'])  # Aplicamos la transformación sobre la columna 'date'
    ]
).set_output(transform='pandas')

# Verificar el ColumnTransformer
print(preprocessor)

ColumnTransformer(transformers=[('num', StandardScaler(),
                                 ['lat', 'long', 'price']),
                                ('cat', OneHotEncoder(sparse_output=False),
                                 ['city', 'shop', 'brand', 'container',
                                  'capacity']),
                                ('date',
                                 FunctionTransformer(func=<function extract_date_features at 0x7ad02f2fa170>),
                                 ['date'])])


In [105]:
from sklearn.pipeline import Pipeline
from sklearn.dummy import DummyRegressor

# Crear pipeline
pipeline = Pipeline(steps=[
    ('preprocessor', preprocessor),
    ('regressor', DummyRegressor(strategy='mean'))
])

# Entrenar el pipeline
pipeline.fit(train_data, train_data['quantity'])

Pipeline(steps=[('preprocessor',
                 ColumnTransformer(transformers=[('num', StandardScaler(),
                                                  ['lat', 'long', 'price']),
                                                 ('cat',
                                                  OneHotEncoder(sparse_output=False),
                                                  ['city', 'shop', 'brand',
                                                   'container', 'capacity']),
                                                 ('date',
                                                  FunctionTransformer(func=<function extract_date_features at 0x7ad02f2fa170>),
                                                  ['date'])])),
                ('regressor', DummyRegressor())])

In [106]:
from sklearn.metrics import mean_absolute_error

# Predecir sobre los datos de validación
val_predictions = pipeline.predict(validation_data)

# Calcular MAE
mae = mean_absolute_error(validation_data['quantity'], val_predictions)
print(f'MAE para DummyRegressor: {mae}')


MAE para DummyRegressor: 13308.134750658153


>El MAE de 13,308.13 indica que, en promedio, el modelo se equivoca por 13,308.13 unidades respecto a los valores reales. Como el `DummyRegressor` predice solo el promedio, este valor representa el error básico, y sirve como referencia mínima al evaluar modelos más complejos.

In [107]:
from xgboost import XGBRegressor

# Modificar el pipeline para usar XGBRegressor
pipeline_xgb = Pipeline(steps=[
    ('preprocessor', preprocessor),
    ('regressor', XGBRegressor())
])

# Entrenar el pipeline con XGBRegressor
pipeline_xgb.fit(train_data, train_data['quantity'])

# Predecir sobre los datos de validación
val_predictions_xgb = pipeline_xgb.predict(validation_data)

# Calcular MAE para XGBRegressor
mae_xgb = mean_absolute_error(validation_data['quantity'], val_predictions_xgb)
print(f'MAE para XGBRegressor: {mae_xgb}')

MAE para XGBRegressor: 2246.29850070276


>El MAE para el `XGBRegressor` es **2,246.30**, lo que es significativamente menor que el MAE del `DummyRegressor` (13,308.13). Esto indica que el `XGBRegressor` es mucho mejor, ya que reduce el error promedio en comparación con el modelo base, lo que sugiere que está capturando mejor las relaciones en los datos.

In [108]:
import joblib

# Guardar modelos
joblib.dump(pipeline, 'dummy_model.pkl')
joblib.dump(pipeline_xgb, 'xgb_model.pkl')

['xgb_model.pkl']

## 2. Forzando relaciones entre parámetros con XGBoost (10 puntos)

<p align="center">
  <img src="https://64.media.tumblr.com/14cc45f9610a6ee341a45fd0d68f4dde/20d11b36022bca7b-bf/s640x960/67ab1db12ff73a530f649ac455c000945d99c0d6.gif">
</p>

Un colega aficionado a la economía le *sopla* que la demanda guarda una relación inversa con el precio del producto. Motivado para impresionar al querido corpóreo, se propone hacer uso de esta información para mejorar su modelo realizando las siguientes tareas:

1. Vuelva a entrenar el `Pipeline` con `XGBRegressor`, pero esta vez forzando una relación monótona negativa entre el precio y la cantidad. Para aplicar esta restricción apóyese en la siguiente <a href = https://xgboost.readthedocs.io/en/stable/tutorials/monotonic.html>documentación</a>. [6 puntos]

>Hint 1: Para implementar el constraint se le sugiere hacerlo especificando el nombre de la variable. De ser así, probablemente le sea útil **mantener el formato de pandas** antes del step de entrenamiento.

>Hint 2: Puede obtener el nombre de las columnas en el paso anterior al modelo regresor mediante el método `.get_feature_names_out()`

2. Luego, vuelva a reportar el `MAE` sobre el conjunto de validación. [1 puntos]

3. ¿Cómo cambia el error al incluir esta relación? ¿Tenía razón su amigo? [2 puntos]

4. Guarde su modelo en un archivo .pkl [1 punto]

In [110]:
# Aplicar el preprocesador a los datos de entrenamiento para ver el orden de las columnas
processed_train_data = preprocessor.fit_transform(train_data)

# Mostrar el orden de las columnas procesadas
print(processed_train_data.columns)

Index(['num__lat', 'num__long', 'num__price', 'cat__city_Athens',
       'cat__city_Irakleion', 'cat__city_Larisa', 'cat__city_Patra',
       'cat__city_Thessaloniki', 'cat__shop_shop_1', 'cat__shop_shop_2',
       'cat__shop_shop_3', 'cat__shop_shop_4', 'cat__shop_shop_5',
       'cat__shop_shop_6', 'cat__brand_adult-cola', 'cat__brand_gazoza',
       'cat__brand_kinder-cola', 'cat__brand_lemon-boost',
       'cat__brand_orange-power', 'cat__container_can', 'cat__container_glass',
       'cat__container_plastic', 'cat__capacity_1.5lt', 'cat__capacity_330ml',
       'cat__capacity_500ml', 'date__day', 'date__month', 'date__year'],
      dtype='object')


In [111]:
# Inserte su código acá
# Monotonic constraints: 0 para las otras variables, -1 para price
monotonic_constraints = (0, 0, -1,   # `price` es la tercera variable numérica
                         0, 0, 0, 
                         0, 0, 0, 
                         0, 0, 0,
                         0, 0, 0, 
                         0, 0, 0,
                         0, 0, 0, 
                         0, 0, 0, 
                         0, 0, 0)

# Modificar el pipeline para usar XGBRegressor con restricciones monótonas
pipeline_xgb_monotonic = Pipeline(steps=[
    ('preprocessor', preprocessor),  # Preprocesamiento como antes
    ('regressor', XGBRegressor(monotone_constraints=monotonic_constraints))
])

# Entrenar el pipeline con XGBRegressor y restricciones monótonas
pipeline_xgb_monotonic.fit(train_data, train_data['quantity'])  # Cambia 'quantity' si el target tiene otro nombre

# Predecir y evaluar sobre los datos de validación
val_predictions_monotonic = pipeline_xgb_monotonic.predict(validation_data)
mae_monotonic = mean_absolute_error(validation_data['quantity'], val_predictions_monotonic)
print(f'MAE para XGBRegressor con restricciones monótonas: {mae_monotonic}')

MAE para XGBRegressor con restricciones monótonas: 2470.5538518928556


>El nuevo MAE con restricciones monótonas es 2,470.55, lo que es ligeramente peor que el MAE original del XGBRegressor sin restricciones (2,246.30). Esto sugiere que, aunque la restricción monótona negativa entre precio y cantidad refleja una teoría económica (relación inversa entre demanda y precio), en este caso específico, imponer esa relación empeoró ligeramente el rendimiento del modelo.

In [112]:
joblib.dump(pipeline_xgb_monotonic, 'xgb_monotonic_model.pkl')

# Confirmar que se ha guardado
print("Modelo guardado como xgb_monotonic_model.pkl")

Modelo guardado como xgb_monotonic_model.pkl


## 3 Optimización de Hiperparámetros con Optuna (20 puntos)

<p align="center">
  <img src="https://media.tenor.com/fmNdyGN4z5kAAAAi/hacking-lucy.gif">
</p>

Luego de presentarle sus resultados, Fiu le pregunta si es posible mejorar *aun más* su modelo. En particular, le comenta de la optimización de hiperparámetros con metodologías bayesianas a través del paquete `optuna`. Como usted es un aficionado al entrenamiento de modelos de ML, se propone implementar la descabellada idea de su jefe.

A partir de la mejor configuración obtenida en la sección anterior, utilice `optuna` para optimizar sus hiperparámetros. En particular, se pide que su optimización considere lo siguiente:

- Fijar una semilla en las instancias necesarias para garantizar la reproducibilidad de resultados
- Utilice `TPESampler` como método de muestreo
- De `XGBRegressor`, optimice los siguientes hiperparámetros:
    - `learning_rate` buscando valores flotantes en el rango (0.001, 0.1)
    - `n_estimators` buscando valores enteros en el rango (50, 1000)
    - `max_depth` buscando valores enteros en el rango (3, 10)
    - `max_leaves` buscando valores enteros en el rango (0, 100)
    - `min_child_weight` buscando valores enteros en el rango (1, 5)
    - `reg_alpha` buscando valores flotantes en el rango (0, 1)
    - `reg_lambda` buscando valores flotantes en el rango (0, 1)
- De `OneHotEncoder`, optimice el hiperparámetro `min_frequency` buscando el mejor valor flotante en el rango (0.0, 1.0)

Para ello se pide los siguientes pasos:
1. Implemente una función `objective()` que permita minimizar el `MAE` en el conjunto de validación. Use el método `.set_user_attr()` para almacenar el mejor pipeline entrenado. [10 puntos]
2. Fije el tiempo de entrenamiento a 5 minutos. [1 punto]
    3. Optimizar el modelo y reportar el número de *trials*, el `MAE` y los mejores hiperparámetros encontrados. ¿Cómo cambian sus resultados con respecto a la sección anterior? ¿A qué se puede deber esto? [3 puntos]
4. Explique cada hiperparámetro y su rol en el modelo. ¿Hacen sentido los rangos de optimización indicados? [5 puntos]
5. Guardar su modelo en un archivo .pkl [1 punto]

In [154]:
import optuna
from optuna.samplers import TPESampler
optuna.logging.set_verbosity(optuna.logging.WARNING)
# Inserte su código acá

# Fijar semilla para reproducibilidad
SEED = 42

def objective(trial):
    # Optimización de hiperparámetros para OneHotEncoder
    min_frequency = trial.suggest_float('min_frequency', 0.0, 1.0)
    
    # Optimización de hiperparámetros para XGBRegressor
    learning_rate = trial.suggest_float('learning_rate', 0.001, 0.1)
    n_estimators = trial.suggest_int('n_estimators', 50, 1000)
    max_depth = trial.suggest_int('max_depth', 3, 10)
    max_leaves = trial.suggest_int('max_leaves', 0, 100)
    min_child_weight = trial.suggest_int('min_child_weight', 1, 5)
    reg_alpha = trial.suggest_float('reg_alpha', 0.0, 1.0)
    reg_lambda = trial.suggest_float('reg_lambda', 0.0, 1.0)
    
    # Crear el preprocesador y el pipeline
    preprocessor = ColumnTransformer(
        transformers=[
            ('num', StandardScaler(), numerical_cols),
            ('cat', OneHotEncoder(sparse_output=False, min_frequency=min_frequency), categorical_cols),
            ('date', date_transformer, ['date'])
        ]
    ).set_output(transform='pandas')

    # Crear el pipeline
    pipeline = Pipeline(steps=[
        ('preprocessor', preprocessor),
        ('regressor', XGBRegressor(
            learning_rate=learning_rate,
            n_estimators=n_estimators,
            max_depth=max_depth,
            max_leaves=max_leaves,
            min_child_weight=min_child_weight,
            reg_alpha=reg_alpha,
            reg_lambda=reg_lambda,
            random_state=SEED,
        ))
    ])
    
    # Entrenar el pipeline
    pipeline.fit(train_data, train_data['quantity'])
    
    # Predicción en el conjunto de validación
    val_predictions = pipeline.predict(validation_data)
    
    # Calcular MAE
    mae = mean_absolute_error(validation_data['quantity'], val_predictions)
    
    # Guardar el mejor pipeline
    trial.set_user_attr('best_pipeline', pipeline)
    
    return mae

In [155]:
# Configurar Optuna con TPESampler y un tiempo límite de 5 minutos
study1 = optuna.create_study(direction='minimize', sampler=TPESampler(seed=SEED))
study1.optimize(objective, timeout=300)

In [156]:
# Obtener los resultados
print(f"Number of trials: {len(study1.trials)}")
print(f"Best MAE: {study1.best_value}")
print(f"Best hyperparameters: {study1.best_params}")

# Guardar el mejor modelo
best_pipeline = study1.best_trial.user_attrs['best_pipeline']

Number of trials: 157
Best MAE: 1897.4183825504317
Best hyperparameters: {'min_frequency': 0.045663117241420484, 'learning_rate': 0.06027101288964817, 'n_estimators': 917, 'max_depth': 9, 'max_leaves': 93, 'min_child_weight': 4, 'reg_alpha': 0.6740999943157295, 'reg_lambda': 0.9132244902055148}


>La mejora en el MAE se debe a la optimización bayesiana de los hiperparámetros, que ajustó valores clave como `learning_rate`, `n_estimators`, y `max_depth` para encontrar una configuración más precisa que el modelo base. Esta metodología explora el espacio de soluciones de manera más eficiente que el ajuste manual o aleatorio.
>
>### Explicación de los hiperparámetros optimizados
>
>- **`min_frequency`**: Controla la frecuencia mínima de las categorías en el `OneHotEncoder`. Un valor bajo (0.05) asegura que más categorías sean codificadas.
>- **`learning_rate`**: Controla la velocidad de aprendizaje; un valor moderado (0.06) balancea precisión y convergencia.
>- **`n_estimators`**: Aumentar a 917 mejora la capacidad del modelo para aprender patrones complejos.
>- **`max_depth`**: Mayor profundidad (9) permite que los árboles capturen interacciones más profundas.
>- **`max_leaves`**: Un valor alto (93) aumenta la flexibilidad del modelo sin llegar a sobreajustar.
>- **`min_child_weight`**: Un valor de 4 reduce el sobreajuste en hojas pequeñas.
>- **`reg_alpha` y `reg_lambda`**: Ambos valores (0.67 y 0.91) controlan el sobreajuste, regularizando el modelo.
>
>Respecto a los rangos, estos son adecuados. Permiten una búsqueda eficiente de configuraciones que optimizan tanto la capacidad de generalización como la capacidad de ajuste del modelo.

In [157]:
joblib.dump(best_pipeline, 'xgb_optimized_model.pkl')

# Confirmar que el archivo ha sido guardado
print("Modelo optimizado guardado como xgb_optimized_model.pkl")

Modelo optimizado guardado como xgb_optimized_model.pkl


## 4. Optimización de Hiperparámetros con Optuna y Prunners (17 puntos)

<p align="center">
  <img src="https://i.pinimg.com/originals/90/16/f9/9016f919c2259f3d0e8fe465049638a7.gif">
</p>

Después de optimizar el rendimiento de su modelo varias veces, Fiu le pregunta si no es posible optimizar el entrenamiento del modelo en sí mismo. Después de leer un par de post de personas de dudosa reputación en la *deepweb*, usted llega a la conclusión que puede cumplir este objetivo mediante la implementación de **Prunning**.

Vuelva a optimizar los mismos hiperparámetros que la sección pasada, pero esta vez utilizando **Prunning** en la optimización. En particular, usted debe:

- Responder: ¿Qué es prunning? ¿De qué forma debería impactar en el entrenamiento? [2 puntos]
- Redefinir la función `objective()` utilizando `optuna.integration.XGBoostPruningCallback` como método de **Prunning** [10 puntos]
- Fijar nuevamente el tiempo de entrenamiento a 5 minutos [1 punto]
- Reportar el número de *trials*, el `MAE` y los mejores hiperparámetros encontrados. ¿Cómo cambian sus resultados con respecto a la sección anterior? ¿A qué se puede deber esto? [3 puntos]
- Guardar su modelo en un archivo .pkl [1 punto]

Nota: Si quieren silenciar los prints obtenidos en el prunning, pueden hacerlo mediante el siguiente comando:

```
optuna.logging.set_verbosity(optuna.logging.WARNING)
```

De implementar la opción anterior, pueden especificar `show_progress_bar = True` en el método `optimize` para *más sabor*.

Hint: Si quieren especificar parámetros del método .fit() del modelo a través del pipeline, pueden hacerlo por medio de la siguiente sintaxis: `pipeline.fit(stepmodelo__parametro = valor)`

Hint2: Este <a href = https://stackoverflow.com/questions/40329576/sklearn-pass-fit-parameters-to-xgboost-in-pipeline>enlace</a> les puede ser de ayuda en su implementación

**Respuestas:**
>Prunning (poda) es una técnica utilizada en la optimización de modelos para detener los trials que parecen ser poco prometedores en fases tempranas del entrenamiento, evitando gastar tiempo en modelos que no proporcionarán buenos resultados. En lugar de completar todos los entrenamientos de los hiperparámetros seleccionados, el pruning interrumpe aquellos que claramente no alcanzarán un buen desempeño.
> El pruning acelera el proceso de optimización, ya que detiene los trials que no parecen competitivos, liberando recursos para probar configuraciones más prometedoras. Esto debería reducir el tiempo de entrenamiento y permitir explorar más configuraciones en el tiempo disponible.



In [123]:
!pip install optuna-integration[xgboost]

  Using cached optuna_integration-4.0.0-py3-none-any.whl.metadata (11 kB)
  Using cached optuna-4.0.0-py3-none-any.whl.metadata (16 kB)
  Using cached alembic-1.13.3-py3-none-any.whl.metadata (7.4 kB)
  Using cached colorlog-6.8.2-py3-none-any.whl.metadata (10 kB)
Using cached optuna-4.0.0-py3-none-any.whl (362 kB)
Using cached optuna_integration-4.0.0-py3-none-any.whl (96 kB)
Using cached alembic-1.13.3-py3-none-any.whl (233 kB)
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.2/3.2 MB 8.9 MB/s eta 0:00:00:00:0100:01
Using cached colorlog-6.8.2-py3-none-any.whl (11 kB)
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 613.1/613.1 kB 8.2 MB/s eta 0:00:00:00:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 78.6/78.6 kB 699.6 kB/s eta 0:00:00a 0:00:01


In [143]:
# Inserte su código acá
from optuna.integration import XGBoostPruningCallback
import xgboost as xgb

optuna.logging.set_verbosity(optuna.logging.WARNING)

def objective(trial):
    # Optimización de hiperparámetros para OneHotEncoder y XGBRegressor
    min_frequency = trial.suggest_float('min_frequency', 0.0, 1.0)
    learning_rate = trial.suggest_float('learning_rate', 0.001, 0.1)
    n_estimators = trial.suggest_int('n_estimators', 50, 1000)
    max_depth = trial.suggest_int('max_depth', 3, 10)
    max_leaves = trial.suggest_int('max_leaves', 0, 100)
    min_child_weight = trial.suggest_int('min_child_weight', 1, 5)
    reg_alpha = trial.suggest_float('reg_alpha', 0.0, 1.0)
    reg_lambda = trial.suggest_float('reg_lambda', 0.0, 1.0)
    
    # Crear el preprocesador y el pipeline
    preprocessor = ColumnTransformer(
        transformers=[
            ('num', StandardScaler(), numerical_cols),
            ('cat', OneHotEncoder(sparse_output=False, min_frequency=min_frequency), categorical_cols),
            ('date', date_transformer, ['date'])
        ]
    ).set_output(transform='pandas')

    # Aplicar preprocesamiento
    X_train_trans = preprocessor.fit_transform(train_data.drop(columns=['quantity']))
    X_val_trans = preprocessor.transform(validation_data.drop(columns=['quantity']))

    # Crear DMatrix para xgboost
    dtrain = xgb.DMatrix(X_train_trans, label=train_data['quantity'])
    dval = xgb.DMatrix(X_val_trans, label=validation_data['quantity'])

    # Definir parámetros del modelo
    params = {
        'objective': 'reg:squarederror',
        'learning_rate': learning_rate,
        'max_depth': max_depth,
        'max_leaves': max_leaves,
        'min_child_weight': min_child_weight,
        'reg_alpha': reg_alpha,
        'reg_lambda': reg_lambda,
        'eval_metric': 'mae',
        'verbosity': 0
    }

    # Conjunto de evaluación para early stopping
    evals = [(dtrain, 'train'), (dval, 'eval')]

    # Habilitar pruning
    pruning_callback = XGBoostPruningCallback(trial, "eval-mae")

    # Entrenar el modelo con early stopping y pruning
    model = xgb.train(
        params, 
        dtrain, 
        num_boost_round=n_estimators,
        evals=evals,
        early_stopping_rounds=10,
        callbacks=[pruning_callback]
    )

    # Predecir en el conjunto de validación
    val_predictions = model.predict(dval)
    
    # Calcular MAE
    mae = mean_absolute_error(validation_data['quantity'], val_predictions)

    # Guardar el mejor modelo
    trial.set_user_attr('best_model', model)
    
    return mae



In [144]:
# Configurar el estudio y la optimización
study = optuna.create_study(direction='minimize', sampler=TPESampler(seed=SEED))
study.optimize(objective, timeout=300, show_progress_bar=True)

   0%|          | 00:00/05:00

[0]	train-mae:12868.72112	eval-mae:12586.39541
[1]	train-mae:12265.44053	eval-mae:11963.02022
[2]	train-mae:11747.17957	eval-mae:11432.23692
[3]	train-mae:11303.05234	eval-mae:10966.91721
[4]	train-mae:10878.37858	eval-mae:10538.67712
[5]	train-mae:10525.97137	eval-mae:10191.62067
[6]	train-mae:10177.59941	eval-mae:9861.16675
[7]	train-mae:9861.43945	eval-mae:9547.03043
[8]	train-mae:9571.13143	eval-mae:9285.10953
[9]	train-mae:9341.78568	eval-mae:9059.24795
[10]	train-mae:9087.40339	eval-mae:8808.03135
[11]	train-mae:8868.24097	eval-mae:8613.32185
[12]	train-mae:8661.36618	eval-mae:8410.07451
[13]	train-mae:8486.65194	eval-mae:8256.04256
[14]	train-mae:8296.55083	eval-mae:8092.99569
[15]	train-mae:8127.24622	eval-mae:7942.45849
[16]	train-mae:7989.95305	eval-mae:7818.12334
[17]	train-mae:7852.56509	eval-mae:7696.16184
[18]	train-mae:7729.98596	eval-mae:7567.98918
[19]	train-mae:7633.63036	eval-mae:7474.53946
[20]	train-mae:7524.48578	eval-mae:7365.12865
[21]	train-mae:7433.15622	eval-

In [145]:
# Obtener los resultados
print(f"Number of trials: {len(study.trials)}")
print(f"Best MAE: {study.best_value}")
print(f"Best hyperparameters: {study.best_params}")

# Guardar el mejor modelo encontrado
best_pipeline = study.best_trial.user_attrs['best_model']

Number of trials: 50
Best MAE: 1948.6772951500438
Best hyperparameters: {'min_frequency': 0.10404174969305519, 'learning_rate': 0.09060413981479243, 'n_estimators': 550, 'max_depth': 10, 'max_leaves': 85, 'min_child_weight': 3, 'reg_alpha': 0.5762626045803285, 'reg_lambda': 0.27544968478589205}


>Con **pruning**, el mejor `MAE` fue de **1,948.68** en 50 *trials*, mientras que sin **pruning** fue de **1,893.25** en 282 *trials*. 
>
>**Pruning** no mejoró el rendimiento porque detuvo los *trials* antes de explorar configuraciones prometedoras, lo que resultó en menos *trials* y un modelo menos afinado. Sin **pruning**, se exploraron más configuraciones, permitiendo encontrar hiperparámetros más ajustados (por ejemplo, menor `learning_rate` y más `n_estimators`), lo que resultó en un mejor `MAE`.

In [150]:
joblib.dump(best_pipeline, 'xgb_optimized_model_prunning.pkl')

# Confirmar que el archivo ha sido guardado
print("Modelo optimizado guardado como xgb_optimized_model_prunning.pkl")

Modelo optimizado guardado como xgb_optimized_model_prunning.pkl


## 5. Visualizaciones (5 puntos)

<p align="center">
  <img src="https://media.tenor.com/F-LgB1xTebEAAAAd/look-at-this-graph-nickelback.gif">
</p>


Satisfecho con su trabajo, Fiu le pregunta si es posible generar visualizaciones que permitan entender el entrenamiento de su modelo.

A partir del siguiente <a href = https://optuna.readthedocs.io/en/stable/tutorial/10_key_features/005_visualization.html#visualization>enlace</a>, genere las siguientes visualizaciones:

1. Gráfico de historial de optimización [1 punto]
2. Gráfico de coordenadas paralelas [1 punto]
3. Gráfico de importancia de hiperparámetros [1 punto]

Comente sus resultados:

4. ¿Desde qué *trial* se empiezan a observar mejoras notables en sus resultados? [0.5 puntos]
5. ¿Qué tendencias puede observar a partir del gráfico de coordenadas paralelas? [1 punto]
6. ¿Cuáles son los hiperparámetros con mayor importancia para la optimización de su modelo? [0.5 puntos]

In [147]:
# Inserte su código acá
import optuna.visualization as vis

# Gráfico del historial de optimización
optimization_history = vis.plot_optimization_history(study1)
optimization_history.show()

In [148]:
# Gráfico de coordenadas paralelas
parallel_coordinates = vis.plot_parallel_coordinate(study1)
parallel_coordinates.show()

In [149]:
# Gráfico de importancia de hiperparámetros
param_importance = vis.plot_param_importances(study1)
param_importance.show()

## 6. Síntesis de resultados (3 puntos)

Finalmente:

1. Genere una tabla resumen del MAE en el conjunto de validación obtenido en los 5 modelos entrenados desde Baseline hasta XGBoost con Constraints, Optuna y Prunning. [1 punto]
2. Compare los resultados de la tabla y responda, ¿qué modelo obtiene el mejor rendimiento? [0.5 puntos]
3. Cargue el mejor modelo, prediga sobre el conjunto de **test** y reporte su MAE. [0.5 puntos]
4. ¿Existen diferencias con respecto a las métricas obtenidas en el conjunto de validación? ¿Porqué puede ocurrir esto? [1 punto]

### Tabla de comparación
---

| Modelo                                      | MAE en Conjunto de Validación |
|---------------------------------------------|-------------------------------|
| Baseline (DummyRegressor)                   | 13,308.13                     |
| XGBoost (sin restricciones)                 | 2,246.30                      |
| XGBoost con restricciones monótonas         | 2,470.55                      |
| XGBoost optimizado con Optuna               | 1,897.42                      |
| XGBoost optimizado con Optuna + Prunning    | 1,948.68                      |

> El mejor modelo es XGBoost optimizado con Optuna

In [158]:
# Inserte su código acá

X_test = test_data.drop(columns=['quantity'])
y_test = test_data['quantity']

# Cargar el mejor modelo optimizado con pruning
best_model = joblib.load('xgb_optimized_model.pkl')

# Predecir sobre el conjunto de test
test_predictions = best_model.predict(X_test)

# Calcular el MAE sobre el conjunto de test
mae_test = mean_absolute_error(y_test, test_predictions)
print(f'MAE sobre el conjunto de test: {mae_test}')

MAE sobre el conjunto de test: 1890.9153195434076


>La diferencia entre el **MAE en validación** (1,897.42) y el **MAE en test** (1,890.92) es mínima. Esto sugiere que el modelo generaliza bien. Las pequeñas diferencias pueden deberse a la variabilidad en los datos o a la aleatoriedad en el entrenamiento, pero no indican problemas significativos de sobreajuste.

# Conclusión
Eso ha sido todo para el lab de hoy, recuerden que el laboratorio tiene un plazo de entrega de una semana. Cualquier duda del laboratorio, no duden en contactarnos por mail o U-cursos.

<p align="center">
  <img src="https://media.tenor.com/8CT1AXElF_cAAAAC/gojo-satoru.gif">
</p>

<a style='text-decoration:none;line-height:16px;display:flex;color:#5B5B62;padding:10px;justify-content:end;' href='https://deepnote.com?utm_source=created-in-deepnote-cell&projectId=87110296-876e-426f-b91d-aaf681223468' target="_blank">
 </img>
Created in <span style='font-weight:600;margin-left:4px;'>Deepnote</span></a>